In [ ]:
#Copyright 2018 Amazon.com, Inc. or its affiliates. All Rights Reserved.
#PDX-License-Identifier: MIT-0 (For details, see https://github.com/awsdocs/amazon-rekognition-developer-guide/blob/master/LICENSE-SAMPLECODE.)

import boto3
import json
import time
import cv2
import sys
import io
import os
import numpy as np
from PIL import (
    Image, 
    ImageDraw, 
    ImageFont
)

In [ ]:
def test():
    print("import success")

In [ ]:
def get_object(bucket, file, download):
    client = boto3.client('s3')
    with open(download, 'wb') as f:
        client.download_fileobj(bucket, file, f)

In [ ]:
def upload_to_s3(file, bucket, filename):
    client = boto3.client('s3')
    response = client.upload_file(
        file, 
        bucket, 
        filename
    )
    return response

In [ ]:
def show_bounding_boxes(
    photo_bytes, 
    boxes, 
    colors, 
    labels=None, 
    bg_text=False,
    fnt=None,
    text_fill=(255,255,255,128)
):
    
    image = Image.open(io.BytesIO(photo_bytes))
    draw = ImageDraw.Draw(image)
    
    # auto adjusting font
    if fnt is None:
        fnt_size = int(12 * image.width/800)
        fnt=ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSansMono.ttf", fnt_size)
    
    if labels is None:
        labels = [None]*len(boxes)
    for box, color, label in zip(boxes, colors, labels):
        left = image.width * box['Left']
        top = image.height * box['Top']
        right = (image.width * box['Width']) + left
        bottom = (image.height * box['Height']) + top
        draw.rectangle([left, top, right, bottom], outline=color, width=3)

        if labels[0] is not None:
            pos_y = top if label[1]=="Top" else bottom
            pos_x = left if label[2]=="Left" else right
            
            if bg_text:
                w, h = fnt.getsize(label[0])
                draw.rectangle((pos_x, pos_y, pos_x + w, pos_y + h), fill='black')
            draw.text((pos_x, pos_y), label[0], font=fnt, fill=text_fill)
        
    image.show()

In [ ]:
def show_polygons(image_bytes, polygons, color):
    image = Image.open(io.BytesIO(image_bytes))
    draw = ImageDraw.Draw(image)
    for polygon in polygons:
        draw.polygon([
            (image.width * point['X'], image.height * point['Y']) for point in polygon],
            outline=color)
    image.show()

In [ ]:
def crop_bounding_boxes(
    photo_name,
    photo_bytes, 
    boxes,
    dest_folder="../images/cropped/"
):
    image = Image.open(io.BytesIO(photo_bytes))
    
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
    
    idx = 1
    new_photo_names = []
    for box in boxes:
        left = image.width * box['Left']
        top = image.height * box['Top']
        right = (image.width * box['Width']) + left
        bottom = (image.height * box['Height']) + top
        
        new_photo_name = '.'.join(photo_name.split('.')[:-1]) + '_{}.'.format(idx) + photo_name.split('.')[-1]
        image_cropped = image.crop((left, top, right, bottom))
        image_cropped.save(dest_folder + new_photo_name)
        new_photo_names.append(new_photo_name)
        idx += 1
    
    return new_photo_names

In [ ]:
def concat_horizontal(image1, image2):
    dst = Image.new(
        'RGB', 
        (
            image1.width + image2.width, 
            image1.height
        )
    )
    
    dst.paste(image1, (0, 0))
    dst.paste(image2, (image1.width, 0))
    return dst

In [ ]:
def concat_vertical(image1, image2):
    dst = Image.new(
        'RGB', 
        (
            im1.width, 
            im1.height + image2.height
        )
    )
    dst.paste(image1, (0, 0))
    dst.paste(image2, (0, image1.height))
    return dst

In [ ]:
def video_to_frames(video):
    vid_name = ".".join(video.split(".")[:-1])
    frame_dir = "../videos/{}_frames/".format(vid_name)
    
    if not os.path.exists(frame_dir):
        os.mkdir(frame_dir)
        
    framecount = 0
    while(vidcap.isOpened()):
        ret, frame = vidcap.read()
        if ret == False:
            break
        cv2.imwrite(frame_dir + '{}_{}.jpg'.format(vid_name, framecount) ,frame)
        framecount+=1

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
def np_to_cv2(np_arr):
    return cv2.cvtColor(np_arr, cv2.COLOR_RGBA2RGB)

In [1]:
def resize(image, width=None, height=None):
    if width is not None and height is None:
        ratio = width/image.size[0]
        height = int(ratio * image.size[1])
    elif width is None and height is not None:
        ratio = height/image.size[1]
        width = int(ratio * image.size[0])
    elif width is None and height is None:
        return None
        
    image = image.resize((width, height))
    
    return image
        